In [2]:
!pip install pydicom numpy scikit-image pillow scipy SimpleITK

  Using cached pydicom-3.0.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached simpleitk-2.5.3-cp311-abi3-win_amd64.whl.metadata (7.3 kB)
Using cached pydicom-3.0.1-py3-none-any.whl (2.4 MB)
Using cached simpleitk-2.5.3-cp311-abi3-win_amd64.whl (18.8 MB)

   ---------------------------------------- 0/2 [SimpleITK]
   ---------------------------------------- 0/2 [SimpleITK]
   ---------------------------------------- 0/2 [SimpleITK]
   ---------------------------------------- 0/2 [SimpleITK]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ------------------- 1/2 [pydicom]
   -------------------- ----------------

In [ ]:
import os  
import pydicom  
import numpy as np  
import SimpleITK as sitk  
from PIL import Image  
from skimage import filters, morphology

In [ ]:
def save_as_png(array, path):
    if array.dtype == bool:
        img_array = (array * 255).astype(np.uint8)
    else:
        # Standardize range to 0-255 for visualization
        amin = array.min()
        amax = array.max()
        if amax > amin:
            img_array = ((array - amin) / (amax - amin) * 255).astype(np.uint8)
        else:
            img_array = np.zeros_like(array, dtype=np.uint8)
    
    img = Image.fromarray(img_array)  
    img.save(path)  

In [ ]:
def process_all_steps():
    # Batch function to process DICOMs, saving each preprocessing/segmentation step as PNG in subfolders.
    
    input_dir = "E:/Normal/DICOM"  
    base_output_dir = "E:/Normal/preprocessing_steps"  
    
    steps = [
        "1_Raw",
        "2_HU_Normalized",
        "3_Windowed",
        "4_Denoised",
        "5_Skull_Mask",
        "6_Final_Stripped"
    ]
    
    for step in steps:
        os.makedirs(os.path.join(base_output_dir, step), exist_ok=True)  
    
    files = [f for f in os.listdir(input_dir) if f.endswith('.dcm')]  
    print(f"Processing {len(files)} files into individual steps...")
    
    for f in files:
        try:
            file_path = os.path.join(input_dir, f)
            filename_base = f.replace('.dcm', '.png')
            
            # Step 1: Raw
            # Loads and saves original pixels.
            ds = pydicom.dcmread(file_path)
            raw_pixel = ds.pixel_array
            save_as_png(raw_pixel, os.path.join(base_output_dir, "1_Raw", filename_base))
            
            # Step 2: HU Normalized
            # Applies slope/intercept for HU conversion.
            slope = float(ds.RescaleSlope) if 'RescaleSlope' in ds else 1
            intercept = float(ds.RescaleIntercept) if 'RescaleIntercept' in ds else 0
            hu_image = raw_pixel * slope + intercept
            save_as_png(hu_image, os.path.join(base_output_dir, "2_HU_Normalized", filename_base))
            
            # Step 3: Windowing (Brain Window)
            # Clips HU to brain range (0-80 HU).
            window_center = 40
            window_width = 80
            min_val = window_center - (window_width / 2)
            max_val = window_center + (window_width / 2)
            windowed = np.clip(hu_image, min_val, max_val)
            save_as_png(windowed, os.path.join(base_output_dir, "3_Windowed", filename_base))
            
            # Step 4: Denoised
            # Median filter on normalized windowed image.
            norm_for_filter = ((windowed - min_val) / (max_val - min_val) * 255).astype(np.uint8)
            denoised = filters.median(norm_for_filter, morphology.disk(1))
            save_as_png(denoised, os.path.join(base_output_dir, "4_Denoised", filename_base))
            
            # Step 5: Skull Mask
            # Binary mask from thresholding/morphology.
            thresh = filters.threshold_otsu(denoised)
            binary = denoised > thresh
            from skimage.segmentation import clear_border
            bw = clear_border(binary)
            selem = morphology.disk(2)
            opened = morphology.opening(bw, selem)
            
            from skimage.measure import label, regionprops
            label_img = label(opened)
            props = regionprops(label_img)
            if props:
                largest_region = max(props, key=lambda x: x.area)
                mask = label_img == largest_region.label
                from scipy import ndimage as ndi
                mask = ndi.binary_fill_holes(mask)
                mask = morphology.dilation(mask, morphology.disk(3))
            else:
                mask = np.ones_like(denoised, dtype=bool)
                
            save_as_png(mask, os.path.join(base_output_dir, "5_Skull_Mask", filename_base))
            
            # Step 6: Final Stripped
            # Applies mask to denoised image.
            # Removes skull for brain-only view.
            final = denoised * mask
            save_as_png(final, os.path.join(base_output_dir, "6_Final_Stripped", filename_base))
            
        except Exception as e:
            print(f"Error processing {f}: {e}")

In [7]:
# Run the batch function.
process_all_steps()  # Executes on all files in input_dir, saving steps as PNGs.

Processing 4427 files into individual steps...
